In [1]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula import Tabula 
import pandas as pd
import os
import sys

sys.path.append("/tf/notebooks/Tabula-main")

os.environ["CUDA_VISIBLE_DEVICES"]="3"



# 设置环境变量

import os
%env NCCL_P2P_DISABLE=1
%env NCCL_DEBUG=INFO

%env NCCL_IB_DISABLE=1
# 获取NCCL_IB_DISABLE的环境变量值
nccl_ib_disable = os.getenv('NCCL_IB_DISABLE')

# 打印环境变量的值
print(f"The current value for NCCL_IB_DISABLE is: {nccl_ib_disable}")

/opt/conda/envs/tabula/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


env: NCCL_P2P_DISABLE=1
env: NCCL_DEBUG=INFO
env: NCCL_IB_DISABLE=1
The current value for NCCL_IB_DISABLE is: 1


In [2]:
import pandas as pd

# Or try 'cp1252' encoding
data = pd.read_excel('/tf/notebooks/Tabula-main/Real_Datasets/dataco_final.xlsx')


In [3]:
dataset = data
dataset['Customer Full Name'] = dataset['customer_fname'].astype(str) + dataset['customer_lname'].astype(str)

data = dataset.drop(['customer_email', 'customer_id', 'customer_password', 'customer_fname', 'customer_lname',
      'product_description', 'product_image_url', 'order_zipcode','product_status','order_profit_per_order','product_price','order_city_en','order_city','order_country','customer_street'], axis=1)

data['customer_zipcode'] = data['customer_zipcode'].fillna(0)  # Filling NaN columns with zero

# Time series
data['order_year'] = pd.DatetimeIndex(data['order_date']).year
data['order_month'] = pd.DatetimeIndex(data['order_date']).month
data['order_week_day'] = pd.DatetimeIndex(data['order_date']).day_name()
data['order_hour'] = pd.DatetimeIndex(data['order_date']).hour
# data['order_second'] = pd.DatetimeIndex(data['order date (DateOrders)'])
data['shipping_year'] = pd.DatetimeIndex(data['shipping_date']).year
data['shipping_month'] = pd.DatetimeIndex(data['shipping_date']).month
data['shipping_week_day'] = pd.DatetimeIndex(data['shipping_date']).day_name()
data['shipping_hour'] = pd.DatetimeIndex(data['shipping_date']).hour

label_data = data[['shipping_week_day','order_week_day','payment_type','category_name','customer_city','customer_country','customer_segment','customer_state','department_name','market','order_country_en','order_state','order_status','product_name','shipping_mode','order_region','order_city_corrected']]

data['target']=data['delivery_status']

data = data.drop(['delivery_status'], axis=1)

data


,payment_type,profite_per_order,sales_per_customer,late_delivery_risk,category_id,category_name,customer_city,customer_country,customer_segment,customer_state,...,Customer Full Name,order_year,order_month,order_week_day,order_hour,shipping_year,shipping_month,shipping_week_day,shipping_hour,target
0,DEBIT,91.250000,314.640015,0,73,Sporting Goods,Caguas,Puerto Rico,Consumer,PR,...,CallyHolloway,2018,1,Wednesday,22,2018,2,Saturday,22,Advance shipping
1,TRANSFER,-249.089996,311.359985,1,73,Sporting Goods,Caguas,Puerto Rico,Consumer,PR,...,IreneLuna,2018,1,Saturday,12,2018,1,Thursday,12,Late delivery
2,CASH,-247.779999,309.720001,0,73,Sporting Goods,San Jose,EE. UU.,Consumer,CA,...,GillianMaldonado,2018,1,Saturday,12,2018,1,Wednesday,12,Shipping on time
3,DEBIT,22.860001,304.809998,0,73,Sporting Goods,Los Angeles,EE. UU.,Home Office,CA,...,TanaTate,2018,1,Saturday,11,2018,1,Tuesday,11,Advance shipping
4,PAYMENT,134.210007,298.250000,0,73,Sporting Goods,Caguas,Puerto Rico,Corporate,PR,...,OrliHendricks,2018,1,Saturday,11,2018,1,Monday,11,Advance shipping
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,40.000000,399.980011,0,45,Fishing,Brooklyn,EE. UU.,Home Office,NY,...,MariaPeterson,2016,1,Saturday,3,2016,1,Wednesday,3,Shipping on time
180515,DEBIT,-613.770019,395.980011,1,45,Fishing,Bakersfield,EE. UU.,Corporate,CA,...,RonaldClark,2016,1,Saturday,1,2016,1,Tuesday,1,Late delivery
180516,TRANSFER,141.110001,391.980011,1,45,Fishing,Bristol,EE. UU.,Corporate,CT,...,JohnSmith,2016,1,Friday,21,2016,1,Wednesday,21,Late delivery
180517,PAYMENT,186.229996,387.980011,0,45,Fishing,Caguas,Puerto Rico,Consumer,PR,...,MarySmith,2016,1,Friday,20,2016,1,Monday,20,Advance shipping


In [4]:
import pandas as pd

# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns
print("Categorical columns:", categorical_columns)


Categorical columns: Index(['payment_type', 'category_name', 'customer_city', 'customer_country',
       'customer_segment', 'customer_state', 'department_name', 'market',
       'order_city_corrected', 'order_country_en', 'order_region',
       'order_state', 'order_status', 'product_name', 'shipping_mode',
       'Customer Full Name', 'order_week_day', 'shipping_week_day', 'target'],
      dtype='object')


In [5]:
categorical_columns = ['payment_type', 'category_name', 'customer_city', 'customer_country',
       'customer_segment', 'customer_state', 'department_name', 'market',
       'order_city_corrected', 'order_country_en', 'order_region',
       'order_state', 'order_status', 'product_name', 'shipping_mode',
       'Customer Full Name', 'order_week_day', 'shipping_week_day', 'target']
model = Tabula(llm='distilgpt2', experiment_dir = "insurance_training", batch_size=32, epochs=9, categorical_columns = categorical_columns)

In [6]:
# Comment this block out to test tabula starting from randomly initialized model.
# Comment this block out when uses tabula_middle_padding
import torch
model.model.load_state_dict(torch.load("pretrained-model/adult_king_insurance_intrusion_covtype_pretrained.pt"), strict=False)

/tmp/ipykernel_38195/3909679578.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.model.load_state_dict(torch.load("pretrained-model/adult_king_insurance_intrusion_c

<All keys matched successfully>

In [7]:
# %env CUDA_VISIBLE_DEVICES=3

model.fit(data)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.830300
1000,1.000200
1500,0.902000
2000,0.862600


In [ ]:
import torch
torch.save(model.model.state_dict(), "insurance_training/model_400epoch.pt")

: 

In [ ]:
synthetic_data = model.sample(n_samples=1338)
synthetic_data.to_csv("insurance_400epoch.csv", index=False)

: 

: 